### **Abstract**

In [1]:
from folktables import ACSDataSource, ACSEmployment, BasicProblem, adult_filter
import numpy as np

STATE = "MI"

data_source = ACSDataSource(survey_year='2018', 
                            horizon='1-Year', 
                            survey='person')

acs_data = data_source.get_data(states=[STATE], download=True)

acs_data.head()

,RT,SERIALNO,DIVISION,SPORDER,PUMA,REGION,ST,ADJINC,PWGTP,AGEP,...,PWGTP71,PWGTP72,PWGTP73,PWGTP74,PWGTP75,PWGTP76,PWGTP77,PWGTP78,PWGTP79,PWGTP80
0,P,2018GQ0000064,3,1,2907,2,26,1013097,8,60,...,9,0,12,9,11,9,0,9,10,12
1,P,2018GQ0000154,3,1,1200,2,26,1013097,92,20,...,92,91,93,95,93,173,91,15,172,172
2,P,2018GQ0000158,3,1,2903,2,26,1013097,26,54,...,26,52,3,25,25,28,28,50,51,25
3,P,2018GQ0000174,3,1,1801,2,26,1013097,86,20,...,85,12,87,12,87,85,157,86,86,86
4,P,2018GQ0000212,3,1,2600,2,26,1013097,99,33,...,98,96,98,95,174,175,96,95,179,97


In [2]:
possible_features=['AGEP', 'SCHL', 'MAR', 'RELP', 'DIS', 'ESP', 'CIT', 'MIG', 'MIL', 'ANC', 'NATIVITY', 'DEAR', 'DEYE', 'DREM', 'SEX', 'RAC1P', 'ESR']
acs_data[possible_features].head()

,AGEP,SCHL,MAR,RELP,DIS,ESP,CIT,MIG,MIL,ANC,NATIVITY,DEAR,DEYE,DREM,SEX,RAC1P,ESR
0,60,15.0,5,17,1,NaN,1,1.0,4.0,1,1,2,2,1.0,1,2,6.0
1,20,19.0,5,17,2,NaN,1,1.0,4.0,2,1,2,2,2.0,2,1,6.0
2,54,18.0,3,16,1,NaN,1,1.0,4.0,4,1,2,2,1.0,1,1,6.0
3,20,18.0,5,17,2,NaN,1,1.0,4.0,4,1,2,2,2.0,1,1,6.0
4,33,18.0,5,16,2,NaN,1,3.0,4.0,2,1,2,2,2.0,1,1,6.0


In [3]:
features_to_use = [f for f in possible_features if f not in ["ESR", "RAC1P"]]

In [4]:
EmploymentProblem = BasicProblem(
    features=features_to_use,
    target='ESR',
    target_transform=lambda x: x == 1,
    group='RAC1P',
    preprocess=lambda x: x,
    postprocess=lambda x: np.nan_to_num(x, -1),
)

features, label, group = EmploymentProblem.df_to_numpy(acs_data)

In [5]:
for obj in [features, label, group]:
  print(obj.shape)

(99419, 15)
(99419,)
(99419,)


In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test, group_train, group_test = train_test_split(
    features, label, group, test_size=0.2, random_state=0)

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix

model = make_pipeline(StandardScaler(), LogisticRegression())
model.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('logisticregression', LogisticRegression())])

In [8]:
y_hat = model.predict(X_test)

In [9]:
(y_hat == y_test).mean()

np.float64(0.7863608931804466)

In [10]:
(y_hat == y_test)[group_test == 1].mean()

np.float64(0.7875706214689265)

In [11]:
(y_hat == y_test)[group_test == 2].mean()



np.float64(0.7777164920022063)

In [12]:
import pandas as pd
df = pd.DataFrame(X_train, columns = features_to_use)
df["group"] = group_train
df["label"] = y_train

In [13]:
df

,AGEP,SCHL,MAR,RELP,DIS,ESP,CIT,MIG,MIL,ANC,NATIVITY,DEAR,DEYE,DREM,SEX,group,label
0,23.0,21.0,5.0,2.0,2.0,0.0,1.0,1.0,4.0,2.0,1.0,2.0,2.0,2.0,2.0,1,True
1,24.0,18.0,5.0,2.0,2.0,0.0,1.0,3.0,4.0,1.0,1.0,2.0,2.0,2.0,2.0,2,True
2,33.0,13.0,3.0,13.0,1.0,0.0,1.0,1.0,4.0,2.0,1.0,2.0,2.0,1.0,1.0,1,True
3,88.0,19.0,1.0,0.0,1.0,0.0,1.0,1.0,2.0,2.0,1.0,1.0,2.0,2.0,1.0,1,False
4,4.0,2.0,5.0,2.0,2.0,5.0,1.0,3.0,0.0,4.0,1.0,2.0,2.0,0.0,2.0,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79530,50.0,21.0,1.0,1.0,1.0,0.0,1.0,1.0,4.0,4.0,1.0,2.0,2.0,2.0,1.0,2,True
79531,70.0,16.0,1.0,0.0,2.0,0.0,1.0,1.0,4.0,1.0,1.0,2.0,2.0,2.0,1.0,1,False
79532,18.0,16.0,5.0,2.0,2.0,0.0,1.0,1.0,4.0,1.0,1.0,2.0,2.0,2.0,1.0,1,False
79533,78.0,16.0,1.0,1.0,1.0,0.0,1.0,1.0,2.0,2.0,1.0,1.0,2.0,2.0,1.0,1,False


How many individuals are in the dataset?

In [14]:
len(df)

79535

Of ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍these ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍individuals, ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍what ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍proportion ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍have ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍target ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍label ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍equal ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍to ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍1? ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍In ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍employment ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍prediction, ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍these ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍would ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍correspond ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍to ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍employed ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍individuals.

In [15]:
print(df["label"].value_counts())
print(35232/(35232+44303))

label
False    44303
True     35232
Name: count, dtype: int64
0.44297479097252784


44.3%

Of ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍these ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍individuals, ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍how ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍many ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍are ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍in ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍each ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍of ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍the ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍groups?

In [16]:
df["group"].value_counts()

group
1    67415
2     6881
6     2061
9     1922
8      670
3      467
5       95
7       24
Name: count, dtype: int64

In ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍each ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍group, ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍what ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍proportion ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍of ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍individuals ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍have ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍target ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍label ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍equal ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍to ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍1?

In [17]:
(df[df["label"] == True]["group"].value_counts())/df["group"].value_counts()

group
1    0.454825
2    0.345880
6    0.492479
9    0.328824
8    0.444776
3    0.419700
5    0.400000
7    0.458333
Name: count, dtype: float64

Check ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍for ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍intersectional ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍trends ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍by ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍studying ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍the ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍proportion ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍of ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍positive ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍target ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍labels ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍broken ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍out ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍by ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍your ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍chosen ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍group ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍labels ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍and ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍an ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍additional ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍group ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍label. ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍For ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍example, ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍if ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍you ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍chose ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍race ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍(RAC1P) ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍as ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍your ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍group, ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍then ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍you ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍could ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍also ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍choose ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍sex ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍(SEX) ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍and ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍compute ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍the ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍proportion ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍of ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍positive ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍labels ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍by ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍both ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍race ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍and ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍sex. ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍This ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍might ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍be ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍a ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍good ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍opportunity ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍to ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍use ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍a ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍visualization ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍such ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍as ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍a ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍bar ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍chart, ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍e.g. via ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍the ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍seaborn package.

In [43]:
(df[df["label"] == True].groupby("group")["SEX"].value_counts())/df.groupby("group")["SEX"].value_counts()

group  SEX
1      1.0    0.486367
       2.0    0.423627
2      1.0    0.308653
       2.0    0.379828
3      1.0    0.434389
       2.0    0.406504
5      1.0    0.360000
       2.0    0.444444
6      1.0    0.573034
       2.0    0.419593
7      1.0    0.400000
       2.0    0.500000
8      1.0    0.495575
       2.0    0.392749
9      1.0    0.323656
       2.0    0.333669
Name: count, dtype: float64

Next step is to make a plot where the x axis is race, y is percentage positive labels, and then the hue of the bar would be indicative of sex. From the pure numbers, we can see that group 2, which represents African Americans in the dataset, is much lower than many of the other groups.

In [ ]:
import seaborn as sns
sns.barplot(x = "group", y = , data = df)

IndexError: boolean index did not match indexed array along axis 0; size of axis is 0 but size of corresponding boolean axis is 67415